In [222]:
import requests
import pandas as pd 
from datetime import datetime

In [223]:
# Conexão com API

NOTION_API_KEY = 'secret_8WGBmr0sryTfGYxDiRnEZnU3lWxZSjCHuPCFBoBw56V'
headers = {
    'Authorization': f'Bearer {NOTION_API_KEY}',
    'Content-Type': 'application/json',
    'Notion-Version': '2022-06-28',
}

In [224]:
# Substitua 'ID_DO_BANCO_DE_DADOS' pelo ID do banco de dados que você deseja consultar
database_id = 'b22997410633431cb6fc160aa6a95d40'

# Faça a solicitação GET para consultar o banco de dados
response = requests.post(f'https://api.notion.com/v1/databases/{database_id}/query', headers=headers)


In [225]:
# Analise a resposta
if response.status_code == 200:
    response_data = response.json()
    
    if 'results' in response_data:
        # Aplicar a normalização aos dados
        df = pd.json_normalize(response_data, 'results')
        
        # Exibir o DataFrame resultante
        print(df.to_string(index=False))
    else:
        print("A resposta não contém dados de resultado.")
else:
    print(f'Erro ao consultar o banco de dados. Código de status: {response.status_code}')

df.to_excel('C:/Users/Lucas Gomes/Downloads/teste1.xlsx',index=False)


object                                   id             created_time         last_edited_time cover  icon  archived                                                                          url public_url created_by.object                        created_by.id last_edited_by.object                    last_edited_by.id parent.type                   parent.database_id properties.Data.id properties.Data.type  properties.Data.date properties.Texto.id properties.Texto.type                                                                                                                                                                                                                                                                  properties.Texto.rich_text properties.Status.id properties.Status.type          properties.Status.status.id properties.Status.status.name properties.Status.status.color properties.Tags.id properties.Tags.type                                                           prope

In [226]:
def extrair_valores(valores):
    if isinstance(valores, list):
        # Verifique se a lista não está vazia
        if valores:
            return valores[0].get('text', {}).get('content')
    return None

# Aplique a função à coluna 'properties.Nome.title'
df['properties.Nome.title'] = df['properties.Nome.title'].apply(extrair_valores)

In [227]:
def extrair_valores(valores):
    if isinstance(valores, list):
        # Verifique se a lista não está vazia
        if valores:
            return valores[0].get('text', {}).get('content')
    return None

# Aplique a função à coluna 'properties.Nome.title'
df['properties.Texto.rich_text'] = df['properties.Texto.rich_text'].apply(extrair_valores)

In [228]:
# Função para extrair o 'id' e 'name' do dicionário
def extrair_id_name(valores):
    if isinstance(valores, list):
        # Verifique se a lista não está vazia
        if valores:
            id_value = valores[0].get('id')
            name_value = valores[0].get('name')
            return id_value, name_value
    return None, None

# Aplique a função à coluna 'properties.Nome.title'
df[['Tag Id', 'Tag Name']] = df['properties.Tags.multi_select'].apply(extrair_id_name).apply(pd.Series)

In [229]:
# Função para formatar uma única data
def formatar_data(df):
    if isinstance(df, str):
        data_formatada = datetime.strptime(df, "%Y-%m-%dT%H:%M:%S.%f%z")
        return data_formatada.strftime("%Y-%m-%d %H:%M:%S")
    else:
        return None

# Aplicar a função a cada coluna de data
colunas_de_data = ['created_time', 'last_edited_time', 'properties.Data.date.start', 'properties.Data.date.end']
for coluna in colunas_de_data:
    df[coluna] = df[coluna].apply(formatar_data)

In [230]:
columns_of_interest = [
    'url',
    'created_time',
    'last_edited_time',
    'created_by.id',
    'last_edited_by.id',
    'parent.database_id',
    'Tag Id',
    'Tag Name',
    'properties.Nome.title',
    'properties.Status.status.name',
    'properties.Texto.rich_text',
    'properties.Data.date.start',
    'properties.Data.date.end',
    'icon.emoji'


]

# Crie um novo DataFrame com as colunas de interesse
df_filtered = df[columns_of_interest]

In [231]:
df_filtered.to_excel('C:/Users/Lucas Gomes/Downloads/teste.xlsx',index=False)